In [65]:
#Data_STEM_Salaries_NN
#Data is already preprocessed, one_hot_encoder already applied, but not scaled yet
#Import dependencies
#Bring in data from github
#Create training and testing sets
#Apply standard scaler
#

In [66]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import numpy as np
import pandas as pd 
import sklearn as skl
#  Import and read the US_Salary_Preprocessed.csv. Imported as raw data from GitHub since my laptop is incompatible with tensorflow (because it has an amd radeon gpu) 
#  and colab cannot open local/from my computer data files

url = 'https://raw.githubusercontent.com/mgsrichard/Capstone-Project/main/resources/US_Salary_Preprocessed.csv'
salaries_df = pd.read_csv(url)
salaries_df.head()

,yearsofexperience,yearsatcompany,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,...,title_Other,title_Product Designer,title_Product Manager,title_Software Engineer,title_Software Engineering Manager,title_Technical Program Manager,gender_Female,gender_Male,gender_Other,gender_none
0,1.5,1.5,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,5.0,3.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,8.0,0.0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,5.0,3.0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,8.5,8.5,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [68]:
# Split our preprocessed data into our features and target arrays
y = salaries_df.salary_level.values
X = salaries_df.drop(columns=["salary_level"]).values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

In [69]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [70]:
# Define the model - deep neural net and the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 10
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)


# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 10)                320       
                                                                 
 dense_8 (Dense)             (None, 5)                 55        
                                                                 
 dense_9 (Dense)             (None, 1)                 6         
                                                                 
Total params: 381
Trainable params: 381
Non-trainable params: 0
_________________________________________________________________


In [71]:
# Compile the model

nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [72]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
1049/1049 [==============================] - 4s 3ms/step - loss: -3069.8447 - accuracy: 0.0014
Epoch 2/50
1049/1049 [==============================] - 2s 2ms/step - loss: -42518.4023 - accuracy: 0.0014
Epoch 3/50
1049/1049 [==============================] - 2s 2ms/step - loss: -157555.1094 - accuracy: 0.0014
Epoch 4/50
1049/1049 [==============================] - 2s 2ms/step - loss: -365532.8438 - accuracy: 0.0014
Epoch 5/50
1049/1049 [==============================] - 2s 2ms/step - loss: -677969.5625 - accuracy: 0.0014
Epoch 6/50
1049/1049 [==============================] - 2s 2ms/step - loss: -1106094.5000 - accuracy: 0.0014
Epoch 7/50
1049/1049 [==============================] - 2s 2ms/step - loss: -1660351.2500 - accuracy: 0.0014
Epoch 8/50
1049/1049 [==============================] - 2s 2ms/step - loss: -2352005.0000 - accuracy: 0.0014
Epoch 9/50
1049/1049 [==============================] - 2s 2ms/step - loss: -3192652.0000 - accuracy: 0.0014
Epoch 10/50
1049/1049 [====

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

350/350 - 1s - loss: -6.7420e+10 - accuracy: 0.0013 - 517ms/epoch - 1ms/step
Loss: -67419516928.0, Accuracy: 0.0013415615539997816


In [ ]:
#Well ok then. Let's see what happens if we try Keras tuner
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=5), activation=activation, input_dim=number_input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
# Import the kerastuner library

from tensorflow import keras
!pip install -q -U keras-tuner
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2,
    overwrite=True)

     |████████████████████████████████| 133 kB 5.1 MB/s 


In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 01m 22s]
val_accuracy: 0.0013415615539997816

Best val_accuracy So Far: 0.0013415615539997816
Total elapsed time: 00h 24m 49s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get top three model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 6, 'num_layers': 1, 'units_0': 6, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 16, 'num_layers': 4, 'units_0': 21, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0, 'units_1': 1, 'units_2': 1, 'units_3': 1}
{'activation': 'relu', 'first_units': 11, 'num_layers': 4, 'units_0': 6, 'units_1': 26, 'units_2': 26, 'units_3': 16, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
